<a href="https://colab.research.google.com/github/bpremika/Data-Sci-Eng/blob/main/code/Week02_DataPrep/6430246621_Assignment2_TitanicDataPrep_ToStudent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

In [11]:
#Import the libraries
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/hw/titanic_training_dataset_v2.csv')

In [13]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
2,3,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
4,5,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0,373450,8.0500,NaN,S


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     865 non-null    float64
 2   Pclass       828 non-null    float64
 3   Name         866 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        858 non-null    float64
 7   Parch        891 non-null    int64  
 8   Ticket       852 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     803 non-null    object 
dtypes: float64(5), int64(2), object(5)
memory usage: 83.7+ KB


# 1) Load data & review the data

<font color='blue'>Q1) What is the shape of this dataset? (rows & columns)</font>

In [15]:
df.shape

(891, 12)

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

# 2) Drop unqualified variables

*   Drop variables with missing > 50%
*   Drop categorical variables with flat values > 70% (variables with the same value in the same column)

<font color='blue'>Q2) How many columns do we have left?</font>

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

In [16]:
condition1 = df.isnull().sum()/df.shape[0] > 0.5
condition2 = df.apply(lambda col: col.value_counts().max()/df.shape[0] > 0.7)
drop_columns = df.columns[condition1 | condition2]

df.drop(drop_columns, axis=1, inplace=True)
df.shape[1]


10

# 3) Remove all rows with missing target (the variable "Survived")

<font color='blue'>Q3) How many rows do we have left?</font>

In [17]:
df.dropna(subset=['Survived'], inplace=True)
df.shape[0]

865

# 4) Handle outliers

For the variable “Fare”, replace outlier values with the boundary values


*   If value < (Q1 - 1.5IQR), relace with (Q1 - 1.5IQR)
*   If value > (Q3 + 1.5IQR), relace with (Q3 + 1.5IQR)

<font color='blue'>Q4) What is the mean of “Fare” after replacing the outliers (round 2 decimal points)?</font>



In [18]:
Q3, Q1 = np.percentile(df.Fare, [75 ,25])
IQR = Q3 - Q1

df.loc[df.Fare < (Q1 - 1.5*IQR), 'Fare'] = (Q1 - 1.5*IQR)
df.loc[df.Fare > (Q3 + 1.5*IQR), 'Fare'] = (Q3 + 1.5*IQR)

print(df.Fare.mean())

24.038996878612718


* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
# 5) Impute missing value


*   Show the number of missing values in each variable
*   Impute missing values with mean & mode
*   Show the number of missing values again after missing value imputation

<font color='blue'>Q5) Which variable has the largest number of missing values?</font>

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 


In [19]:
print(df.isnull().sum())

# 1) select only numeric columns
numerical_cols = df[df.select_dtypes(include=['float64', 'int64']).columns]
from sklearn.impute import SimpleImputer

num_imp=SimpleImputer(missing_values=np.NaN, strategy='mean')
df[df.select_dtypes(include=['float64', 'int64']).columns]=num_imp.fit_transform(numerical_cols)
df.isnull().sum()

# 2) select only categorical columns
categorical_cols = df[df.select_dtypes(include=['object']).columns]
cat_imp=SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
df[df.select_dtypes(include=['object']).columns]= cat_imp.fit_transform(categorical_cols)
df.isnull().sum()

PassengerId      0
Survived         0
Pclass          63
Name            25
Sex              0
Age            172
SibSp           33
Ticket          39
Fare             0
Embarked        87
dtype: int64


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

# 6) Convert categorical to numeric values



*   Drop the variables “Name” & “Ticket”
*   For the variables “Sex” & “Embarked”, perform the <font color='blue'> dummy </font> coding and drop the first level. Also, drop those original variables (“Sex” & “Embarked”)

<font color='blue'>Q6) How many columns do we have?</font>

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

In [20]:
df.drop(['Name','Ticket'], axis=1, inplace=True)

mapping_dict = {
    "Sex": {
        "male": 0,
        "female": 1,
    },
    "Embarked":{
        "S": 0,
        "C": 1,
        "Q": 2,
    }
}

df = df.replace(mapping_dict)

nominal_columns = ["Sex", "Embarked"]
dummy_df = pd.get_dummies(df[nominal_columns], columns=['Sex','Embarked'], drop_first=True)
df = pd.concat([df, dummy_df], axis=1)
df = df.drop(nominal_columns, axis=1)
df.shape[1]

C:\Users\Admin\AppData\Local\Temp\ipykernel_28348\1343128565.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(mapping_dict)


9

# 7) Partition data



*   Split train/test split with stratification using 70%:30% and random seed with 12345
*   Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

<font color='blue'>Q7) What is a proportion between survived (1) and died (0) in the training data?</font>



In [21]:
from sklearn.model_selection import train_test_split
y = df.pop('Survived')
X = df

X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.30, random_state=12345)
print(X_train.shape)
print(X_test.shape)

total_proportions = y.value_counts()
print(total_proportions/y.shape[0])

train_proportions = y_train.value_counts()
print(train_proportions/y_train.shape[0])

test_proportions = y_test.value_counts()
print(test_proportions/y_test.shape[0])

(605, 8)
(260, 8)
Survived
0.0    0.616185
1.0    0.383815
Name: count, dtype: float64
Survived
0.0    0.616529
1.0    0.383471
Name: count, dtype: float64
Survived
0.0    0.615385
1.0    0.384615
Name: count, dtype: float64
